In [1]:
#@title Install Requirements
!pip install -qU "langchain[google-vertexai]"
!pip install -qU langchain-google-vertexai
!pip install langchain-google-genai
!pip install -qU langchain-core
!pip install -qU pypdf langchain_community
!pip install playwright
!playwright install

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.4/99.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 21.2 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.17 which is incompatible.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.2/437.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 MB 9.6 MB/s eta 0:00:00
164.9 MiB [] 0% 125.5s164.9 MiB [] 0% 52.3s164.9 MiB [] 0% 22.5s164.9 MiB [] 0% 13.4s164.9 MiB [] 0% 7.3s164.9 MiB [] 1% 5.3s164.9 MiB [] 2% 4.3s164.9 MiB [] 2% 3.8s164.9 MiB [] 3% 3.4s164.9 MiB [] 4% 3.1s164.9 MiB [] 5% 2.8s164.9 MiB [] 6% 2.8s164.9 MiB [] 6% 2.9s164.9 MiB [] 7% 2.9s164.9 MiB [] 7% 2.8s164.9 MiB [] 8% 2.7s164.9 MiB [] 9% 2.6s164.9 MiB [] 10% 2.5s164.9 MiB [] 11% 2.4s164.9 MiB [] 12% 2.3s164.9 MiB [] 13% 2.3s164.9 MiB [] 14% 2.2s164.9 MiB [] 15% 2.1s164.9 MiB [] 16% 2.0s164.9 MiB [] 17% 2.0s164.9 MiB [] 18%

In [2]:
#@title Get Google AI API Key
## You need to put your Google AI API key.

import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

Enter your Google AI API key: ··········


In [3]:
#@title Authenticate with Google Cloud and your project ID
# Please put your google cloud project id.

import vertexai
from google.colab import auth

gcp_project_id = 'assignment-2-451115' # @param {type: "string"}       ## PUT GOOGLE CLOUD ASSIGNMENT NAME HERE

auth.authenticate_user(project_id=gcp_project_id)

vertexai.init(project=gcp_project_id)

In [9]:
#@title RAG Train Model
#!pip install -qU "langchain[google-vertexai]"
#!pip install -qU langchain-google-vertexai
#!pip install langchain-google-genai
#!pip install -qU langchain-core
#!pip install -qU pypdf langchain_community
#!pip install playwright
#!playwright install

import requests
import textwrap
from langchain.chat_models import init_chat_model
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import AsyncChromiumLoader
from langchain_community.document_transformers import BeautifulSoupTransformer
# embed the document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
import urllib.request
import pandas as pd
from langchain_core.documents import Document
from langchain_community.document_loaders import PyPDFLoader
# Wrap in LangChain LLM
from langchain_community.llms import HuggingFacePipeline as LC_HF_Pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.llms import HuggingFacePipeline

################################# PUT QUESTION HERE #######################################
question = "What is the rating for Mr. Peabody & Sherman?"


############################## FILES TO RAG TRAIN BELOW ###################################
# 1: Load movies_data_1.txt file
file_path = "./movies_data_1.txt"
txt_loader = TextLoader(file_path)
txt_doc_1 = txt_loader.load()

# 2: Load netflix_2025_released_movies CSV file
csv_2025_released_movies = pd.read_csv("./netflix_2025_released_movies.csv")

csv_doc_2 = []
for _, row in csv_2025_released_movies.iterrows():
    content = f"List of Netflix Movies released in 2025:\n Title: {row.get('Title', '')}\nRelease Date: {row.get('Release Date', '')}\nGenre: {row.get('genre', '')}\nDirector(s): {row.get('Director(s)', '')}\nCast Highlights: {row.get('Cast Highlights', '')}Description: {row.get('Description', '')}\n"
    csv_doc_2.append(Document(page_content=content))

# 3: Load netflix_2024 pdf
pdf_path = "./netflix_2024.pdf"
pdf_loader = PyPDFLoader(pdf_path)
pdf_docs_3 = pdf_loader.load()

# 4: Load april_2025_movies pdf
pdf_path = "./netflix_april_2025.pdf"
pdf_loader = PyPDFLoader(pdf_path)
pdf_docs_4 = pdf_loader.load()

# 5: Load march_2025_movies pdf
pdf_path = "./netflix_march_2025.pdf"
pdf_loader = PyPDFLoader(pdf_path)
pdf_docs_5 = pdf_loader.load()

# 6: Load netflix_movies_1_to_100 CSV file
csv_movies_1_to_100 = pd.read_csv("./netflix_movies_1_to_100.csv")

csv_doc_6 = []
for _, row in csv_movies_1_to_100.iterrows():
    content = f"Netflix Movie Data (Ranked Top 100):\n Rank: {row.get('Rank', '')}\nTitle: {row.get('Title', '')}\nRelease Year: {row.get('Year', '')}\nRotten Tomato Score (Rating): {row.get('Tomatometer', '')}\nCritics Consensus: {row.get('Critics Consensus', '')}\nSynopsis: {row.get('Synopsis', '')}\nCast: {row.get('Starring', '')}\nDirected By: {row.get('Directed By', '')}\nAvailable on: {row.get('Streaming On', '')}\n"
    csv_doc_6.append(Document(page_content=content))

# 7: Load custom (1) CSV file
csv_movies_ratings = pd.read_csv("./custom (1).csv")

csv_doc_7 = []
for _, row in csv_movies_ratings.iterrows():
    content = f"List of Movie Ratings (not limited to just Netflix):\n Title ID: {row.get('tconst', '')}\nPrimary Title: {row.get('primaryTitle', '')}\nOriginal Title: {row.get('originalTitle', '')}\nRelease Year: {row.get('startYear', '')}\nGenres: {row.get('genres', '')}\nAverage Rating: {row.get('averageRating', '')}\nNumber of Votes: {row.get('numVotes', '')}\n"
    csv_doc_7.append(Document(page_content=content))

# 8: Load custom_oscar (1) CSV file
csv_oscar_nominations = pd.read_csv("./custom_oscar (1).csv")

csv_doc_8 = []
for _, row in csv_oscar_nominations.iterrows():
    content = f"List of Oscar Nominated Netflix Movies:\nFilm Year: {row.get('year_film', '')}\nCeremony Year: {row.get('year_ceremony', '')}\nCeremony Number: {row.get('ceremony', '')}\nCategory: {row.get('category', '')}\nCanonical Category: {row.get('canon_category', '')}\nNominee: {row.get('name', '')}\nFilm: {row.get('film', '')}\nWinner: {row.get('winner', '') }\n"
    csv_doc_8.append(Document(page_content=content))



# 10: Load netflix_titles CSV file
csv_netflix_titles = pd.read_csv("./netflix_titles.csv")

csv_doc_10 = []
for _, row in csv_netflix_titles.iterrows():
    content = f"List of Netflix Movies:\nType: {row.get('type', '')}\nTitle: {row.get('title', '')}\nDirector: {row.get('director', '')}\nCast Members: {row.get('cast', '')}\nCountry: {row.get('country', '')}\nDate Added: {row.get('data_added', '')}\nReleased Year: {row.get('release_year', '')}\nRating: {row.get('rating', '') }\nDuration: {row.get('duration', '') }\nCategory: {row.get('listed_in', '') }\nDescription: {row.get('description', '') }\n"
    csv_doc_10.append(Document(page_content=content))

##############################################################################################

# Combine all docs
docs = txt_doc_1 + csv_doc_2 + pdf_docs_3 + pdf_docs_4 +pdf_docs_5 + csv_doc_6 + csv_doc_7 + csv_doc_8 + csv_doc_10


# Split the text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)


# load chat model
llm = init_chat_model("gemini-2.0-flash-001", model_provider="google_vertexai")


# load embedding model
embedding_model = VertexAIEmbeddings(model="text-embedding-004")

# vector store
vector_store = InMemoryVectorStore(embedding=embedding_model)
print(f"Number of chunks to embed: {len(splits)}")
from tqdm import tqdm
vector_store.add_texts([docs.page_content for docs in tqdm(splits)])
#vector_store.add_texts([docs.page_content for docs in splits])
retriever = vector_store.as_retriever(search_kwargs={"k": 10})

# Run without RAG
baseline_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an LLM that should answer questions to the best of your ability."),
    ("human", "{input}"),
])
baseline_chain = baseline_prompt | llm

baseline_response = baseline_chain.invoke({"input": question})

print("\n--- Pre-RAG Output ---")
print(textwrap.fill(baseline_response.content, width=110))



# run inference with RAG
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

system_prompt = (
    "You are an LLM that should answer my questions with correct answers."
    "Please answer the quuestion to the best of your ability. Use both the context and your own knowledge "
    "to answer the question as best you can.\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

results = rag_chain.invoke({"input": question})
output_text = results['answer']
wrapped_text = textwrap.fill(output_text, width=110)
print("\n--- Post-RAG Output ---")
print(wrapped_text)

#print(results['answer'])

Number of chunks to embed: 24561


100%|██████████| 24561/24561 [00:00<00:00, 1695964.91it/s]



--- Pre-RAG Output ---
*Mr. Peabody & Sherman* has the following ratings:  *   **Common Sense Media:** 7+ *   **IMDb:** 6.8/10 *
**Rotten Tomatoes:** 79% *   **Metacritic:** 59/100

--- Post-RAG Output ---
Mr. Peabody & Sherman has several ratings depending on the source:  *   **IMDB:** 6.7 (based on 81,029 votes)
*   **Audience Rating on IMDB (Netflix data):** 6.7 *   **Rotten Tomatoes:** 81%


In [21]:
question = 'What is the highest rated Netflix movie' # @param {type: "string"}
results = rag_chain.invoke({"input": question})
output_text = results['answer']
wrapped_text = textwrap.fill(output_text, width=110)
print("\n--- Post-RAG Output ---")
print(wrapped_text)


--- Post-RAG Output ---
Based on the provided data, the highest-rated Netflix movie is "Schindler's List" with a Rotten Tomato Score
of 98%.
